In [1]:
student_id= 46184821
student_name= 'aditya agarwal'

## BUSA3020 - Assignment 1

---

**Assignment Points**: 100  
**Due Date**: Friday Week 7 (5 April 2024) @ 11.59pm  
**Instructions**: Provide answers within this Jupyter notebook using Python code, and submit via iLearn 
**Marking Criteria**: See end of document below   
**Assignment Background**: This assignment uses a dataset which is based on the Credit Card Defaults data discussed in Week 2 Tutorial  


---

**Problem 1** - Reading the dataset (Total Marks: 20)



**Q1**. Read the first 10,000 rows from the credit card dataset provided in the **assignment_data** folder 
- Name your DataFrame `df` 
- Delete `ID` column     

(5 marks) 




In [2]:
# importing required libraries 

import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [3]:
# naming the dataframe 'df'
df = pd.read_excel('assignment_data/credit_data.xlsx', nrows=10000)

In [4]:
#printing the rows and columns of the dataset
df.shape


(10000, 25)

In [5]:
# deleting the column 'id'
df.drop('ID', axis=1, inplace=True)

<hr style="width:25%;margin-left:0;"> 

**Q2**. List which *features* are *numeric*, *ordinal*, and *nominal* variables, and how many features of each kind there are in the dataset.
To answer this question 
- Find the definitions of the variables provided elsewhere in the course material (hint: make sure you do weekly tutorials)
- Find the definitions of numeric, ordinal and nominal variables
- Carefully consider the values of the data itself as well as the output of `df.info()`. 

Your answer should be written up in Markdown and include:
1) Definitions of the three kinds of variables,
2) A table listing all the features present in the dataset and their type (fill out the table template provided below) and
3) A brief description of the contents of the table.

|Variable Kind|Number of Features|Feature Names
| --- | --- | --- |
| Numeric | some number | some text |
| some text  | some number | some text |
| some text  | some number | some text |


(10 marks)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LIMIT_BAL                   10000 non-null  int64  
 1   SEX                         9617 non-null   float64
 2   EDUCATION                   9617 non-null   float64
 3   MARRIAGE                    9617 non-null   float64
 4   AGE                         9617 non-null   float64
 5   PAY_0                       9617 non-null   float64
 6   PAY_2                       9617 non-null   float64
 7   PAY_3                       9617 non-null   float64
 8   PAY_4                       9642 non-null   float64
 9   PAY_5                       9642 non-null   float64
 10  PAY_6                       9642 non-null   float64
 11  BILL_AMT1                   9642 non-null   float64
 12  BILL_AMT2                   9642 non-null   float64
 13  BILL_AMT3                   9642

## Variable Definitions

1. **Numeric Variables**: These are variables with quantifiable values that can be ordered or ranked. They can be interval scales, which have a meaningful order and a meaningful and constant scale, or ratio scales, which have a true zero point.

2. **Ordinal Variables**: These are categorical variables with a clear order or rank to their values but do not have a constant scale between categories. These variables can often be counted, ranked, or ordered but the differences between the values are not necessarily uniform.

3. **Nominal Variables**: These are categorical variables that represent categories without an intrinsic order or ranking. The categories are typically labels or names for groups, and the numbers are merely identifiers that have no quantitative significance.

## Feature Classification Table

| Variable Kind | Number of Features | Feature Names                              |
|---------------|--------------------|--------------------------------------------|
| Numeric       | 13                 | LIMIT_BAL, AGE, BILL_AMT1 to BILL_AMT6, PAY_AMT1 to PAY_AMT6 |
| Ordinal       | 7                  | EDUCATION, PAY_0 to PAY_6                  |
| Nominal       | 2                  | SEX, MARRIAGE                              |

## Table Description

The table above categorizes the variables in the dataset into three types: Numeric, Ordinal, and Nominal. Numeric variables are those with measurable quantities, while Ordinal variables are those with an inherent order but no constant scale. Nominal variables represent distinct categories without any order. The 'Number of Features' column indicates how many variables fall under each category, and the 'Feature Names' column lists the specific variable names associated with


<hr style="width:25%;margin-left:0;"> 

**Q3.** Missing Values. 

- Print out the number of missing values for each variable in the dataset and comment on your findings.

(5 marks)

In [7]:
# Calculating the number of missing values for each variable
missing_values = df.isnull().sum()

# Printing out the number of missing values for each variable
print(missing_values)

LIMIT_BAL                       0
SEX                           383
EDUCATION                     383
MARRIAGE                      383
AGE                           383
PAY_0                         383
PAY_2                         383
PAY_3                         383
PAY_4                         358
PAY_5                         358
PAY_6                         358
BILL_AMT1                     358
BILL_AMT2                     358
BILL_AMT3                     358
BILL_AMT4                     360
BILL_AMT5                     360
BILL_AMT6                     360
PAY_AMT1                      360
PAY_AMT2                      360
PAY_AMT3                        2
PAY_AMT4                        2
PAY_AMT5                      300
PAY_AMT6                      300
default payment next month      0
dtype: int64




### Comment on Findings:

The dataset presents a number of missing values in various columns. 

- The variables `SEX`, `EDUCATION`, `MARRIAGE`, `AGE`, and payment history columns `PAY_0` to `PAY_3` each have **383 missing values**. This suggests that for a certain subset of records, these particular demographic and payment status details were not captured or are missing.
- A similar pattern of missing data is observed for the variables `PAY_4` to `PAY_6`, `BILL_AMT1` to `BILL_AMT6`, and `PAY_AMT1` to `PAY_AMT2`, which each have between **358 to 360 missing values**.
- There are only **2 missing entries** for `PAY_AMT3` and `PAY_AMT4`, indicating that these might be random omissions.
- There are **300 missing values** for `PAY_AMT5` and `PAY_AMT6`, which could point to a systematic issue in recording these payments.
- There are **no missing values** for the `ID` and `LIMIT_BAL` columns, as well as the target variable `default payment next month`. This indicates complete data in identifying information and credit limit, which are crucial for any financial analysis.


Variables with a high number of missing values may significantly affect analyses and predictive modeling.  The lack of data in key variables may significantly influence the outcomes of any analysis and could lead to biased


---
---

**Problem 2.** Cleaning data and dealing with categorical features (Total Marks: 40)


**Q1.** 

- Use an appropriate `pandas` function to impute missing values using one of the following two strategies: `mean` and `mode`. (10 marks)
    - Take into consideration the type of each variable  and the best practices we discussed in class/lecture notes
- Explain what data imputation is, how you have done it here, and what decisions you had to make. (5 marks)



(Total: 15 marks)

In [8]:
# Imputing missing values for continuous variables using the mean
for column in ['AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
               'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']:
    df[column].fillna(df[column].mean(), inplace=True)

# Imputing missing values for categorical variables using the mode
for column in ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
    df[column].fillna(df[column].mode()[0], inplace=True)


### Data Imputation :
Data imputation is the process of replacing missing data with substituted values. When we impute, our goal is to use existing data to infer missing values while minimizing any distortion in the dataset. The choice of imputation method can depend on the type of data and the nature of the missingness.

### Decisions Made:
For numerical variables, the mean is often used since it can provide a good central tendency estimate, especially if the data is normally distributed. However, if the data contains outliers or is not normally distributed, the median might be a better choice. For categorical variables, the mode (the most frequently occurring value) is typically used, as it represents the most common category.

In this case, considering the dataset involves financial information, decisions on imputation were guided by the nature of the variables:

- For continuous variables, such as `BILL_AMT` and `PAY_AMT`, which represent amounts, the mean is a suitable choice assuming that the data follows a normal distribution and does not contain extreme outliers.
- For categorical variables that are encoded as numbers, such as `EDUCATION` and `MARRIAGE`, the mode is appropriate as it represents the most common category within the data.



1. Identified the type of each variable that contains missing values.
2. Used the mean to impute missing values for continuous variables.
3. Used the mode to impute missing values for categorical variables.


<hr style="width:25%;margin-left:0;"> 

**Q2**. 
- Print `value_counts()` of the 'SEX' column and add a dummy variable named 'SEX_FEMALE' to `df` using `get_dummies()` (3 marks)
- Carefully explain what the values of the new variable 'SEX_FEMALE' mean (2 mark)
- Make sure the variable 'SEX' is deleted from `df`

(Total: 5 marks)  

In [9]:
# Print value counts of the 'SEX' column
sex_value_counts = df['SEX'].value_counts()

# Add a dummy variable 'SEX_FEMALE' to df using get_dummies()
sex_dummies = pd.get_dummies(df['SEX'], prefix='SEX', drop_first=True)
df['SEX_FEMALE'] = sex_dummies

# Delete the 'SEX' variable from df
df.drop('SEX', axis=1, inplace=True)

# Displaying the first few rows to confirm changes
df.head()



,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,SEX_FEMALE
0,310000,3.0,1.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,...,57779.0,14163.0,8295.0,6000.0,4000.0,3000.0,1000.0,2000.0,0,0
1,10000,3.0,1.0,49.0,-1.0,-1.0,-2.0,-1.0,2.0,2.0,...,1138.0,930.0,0.0,0.0,2828.0,0.0,182.0,0.0,1,1
2,50000,2.0,1.0,28.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,...,1300.0,43987.0,0.0,46257.0,2200.0,1300.0,43987.0,1386.0,0,0
3,80000,3.0,1.0,52.0,2.0,2.0,3.0,3.0,3.0,2.0,...,39816.0,40607.0,3700.0,1600.0,1600.0,0.0,1600.0,1600.0,1,1
4,270000,1.0,2.0,34.0,1.0,2.0,0.0,0.0,2.0,0.0,...,15490.0,17343.0,0.0,4000.0,2000.0,0.0,2000.0,2000.0,0,0


### Explation

Before the 'SEX' column was excluded from the DataFrame df, an analysis using value_counts() revealed the presence of 6,028 entries marked as 2.0 and 3,934 entries as 1.0. After generating the dummy variable SEX_FEMALE using the get_dummies() function, the 'SEX' column was removed from df. The new SEX_FEMALE column contains binary values: 0 or 1. Here's what these values mean:

A 0 in the SEX_FEMALE column indicates that the individual's sex was coded as 1 in the original 'SEX' column, which is male.
A 1 in the SEX_FEMALE column indicates that the individual's sex was coded as 2 in the original 'SEX' column, which is female.
The SEX_FEMALE variable, therefore, is a binary indicator for whether the individual is female (1) or not (0).

<hr style="width:25%;margin-left:0;"> 

**Q3**. Print `value_counts()` of the 'MARRIAGE' column and *carefully* comment on what you notice in relation to the definition of this variable. 

(Total: 5 marks) 

In [10]:
# printing value counts of the 'MARRIAGE' column
marriage_value_counts = df['MARRIAGE'].value_counts()

print(marriage_value_counts)


2.0    5511
1.0    4361
3.0     110
0.0      18
Name: MARRIAGE, dtype: int64


### Comments

The value_counts() of the 'MARRIAGE' column in df has: 

2.0 has 5,294 occurrences.
1.0 has 4,537 occurrences.
3.0 has 113 occurrences.
0.0 has 18 occurrences.
Based on common coding conventions in datasets like these, 1 denotes married, 2 represents single, and 3  signifies divorcee, widowed, etc.  It's plausible that 0 could represent missing or undefined data, or possibly another category not detailed in the dataset's documentation.

When analyzing this variable, it's crucial to consider the potential implications of the 0 category. Depending on the dataset's documentation or the context of the study, one might need to decide whether to treat 0 as missing data, group it with another category, or consider it a separate category altogether. This decision could significantly impact any subsequent analyses or models that utilize the 'MARRIAGE' variable. 

<hr style="width:25%;margin-left:0;"> 

**Q4**. 

- Apply `get_dummies()` to 'MARRIAGE' feature and add dummy variables 'MARRIAGE_MARRIED', 'MARRIAGE_SINGLE', 'MARRIAGE_OTHER' to `df`. (5 marks)   
- Carefully consider how to allocate all the values of 'MARRIAGE' across these 3 newly created features (5 marks)   
    - Explain what decisions you had to make
- Make sure that 'MARRIAGE' is deleted from `df`   

(Total: 10 marks)   

In [11]:
 #Step 1: Apply get_dummies() to 'MARRIAGE'
marriage_dummies = pd.get_dummies(df['MARRIAGE'], prefix='MARRIAGE')

# Step 2: Rename and consolidate dummy variables
marriage_dummies = marriage_dummies.rename(columns={'MARRIAGE_1.0': 'MARRIAGE_MARRIED', 
                                                    'MARRIAGE_2.0': 'MARRIAGE_SINGLE'})
marriage_dummies['MARRIAGE_OTHER'] = marriage_dummies[['MARRIAGE_0.0', 'MARRIAGE_3.0']].max(axis=1)

# Remove the now unnecessary original dummy columns for '0.0' and '3.0'
marriage_dummies = marriage_dummies.drop(columns=['MARRIAGE_0.0', 'MARRIAGE_3.0'])

# Step 3: Add the new dummy variables back to df
df = pd.concat([df, marriage_dummies], axis=1)
# Step 4: Remove the 'MARRIAGE' column
df.drop('MARRIAGE', axis=1, inplace=True)

# Displaying the first few rows to confirm changes
df.head()


,LIMIT_BAL,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,SEX_FEMALE,MARRIAGE_MARRIED,MARRIAGE_SINGLE,MARRIAGE_OTHER
0,310000,3.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,172772.0,...,6000.0,4000.0,3000.0,1000.0,2000.0,0,0,1,0,0
1,10000,3.0,49.0,-1.0,-1.0,-2.0,-1.0,2.0,2.0,32.0,...,0.0,2828.0,0.0,182.0,0.0,1,1,1,0,0
2,50000,2.0,28.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,430.0,...,46257.0,2200.0,1300.0,43987.0,1386.0,0,0,1,0,0
3,80000,3.0,52.0,2.0,2.0,3.0,3.0,3.0,2.0,36649.0,...,1600.0,1600.0,0.0,1600.0,1600.0,1,1,1,0,0
4,270000,1.0,34.0,1.0,2.0,0.0,0.0,2.0,0.0,20979.0,...,4000.0,2000.0,0.0,2000.0,2000.0,0,0,0,1,0


### Decisions Made:
To apply get_dummies() to the 'MARRIAGE' feature and add the dummy variables 'MARRIAGE_MARRIED', 'MARRIAGE_SINGLE', and 'MARRIAGE_OTHER' to the DataFrame df, the following steps were taken:

The get_dummies() function was used on the 'MARRIAGE' column to create a set of dummy variables. Each unique value in 'MARRIAGE' became a separate column.

The resulting dummy variables were then carefully renamed to align with the desired new feature names: 'MARRIAGE_MARRIED', 'MARRIAGE_SINGLE', and 'MARRIAGE_OTHER'. Given the original values observed in 'MARRIAGE' (1.0 for married, 2.0 for single, 3.0 for others, and 0.0 potentially as another form of 'others'), decisions made are:

1.0 to 'MARRIAGE_MARRIED'.
2.0 to 'MARRIAGE_SINGLE'.
Combining 3.0 and 0.0 under 'MARRIAGE_OTHER', considering both as representing other marital statuses not explicitly defined as married or single.
After renaming and consolidating the dummy variables, they were added back to df.

Finally, the 'MARRIAGE' column was removed from df, ensuring it only contained the newly defined dummy variables.

##### <hr style="width:25%;margin-left:0;"> 

**Q5**. In the column 'EDUCATION', convert the values {0, 5, 6} to the value 4. 

(Total: 5 marks)  

In [12]:
# Replace the values {0, 5, 6} with 4
df['EDUCATION'] = df['EDUCATION'].replace({0: 4, 5: 4, 6: 4})

---
---

**Problem 3** Preparing X and y arrays (Total Marks: 10)

**Q1**. 

- Create a numpy array `y` from the first 7,500 observations of 'payment_default' column from `df` (2.5 marks)   
- Create a numpy array `X`  from the first 7,500 observations of all the remaining variables in `df` (2.5 marks)   

(Total: 5 Marks)

In [13]:
# creating a numpy array y from the first 7,500 observations of 'payment_default' column from df
y=df.loc[0:7499, 'default payment next month'].values

# creating a numpy array X from the first 7,500 observations of all the remaining variables in df
X= df.loc[0:7499, ['LIMIT_BAL', 'EDUCATION', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
                   'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 
                   'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SEX_FEMALE'
                   , 'MARRIAGE_MARRIED', 'MARRIAGE_SINGLE', 'MARRIAGE_OTHER']].values

<hr style="width:25%;margin-left:0;"> 

**Q2**. 

- Use an appropriate `sklearn` library we used in class to create `y_train`, `y_test`, `X_train` and `X_test` by splitting the data into 75% train and 25% test datasets (2.5 marks) 
    - Set random_state to 3 and stratify the subsamples so that train and test datasets have roughly equal proportions of the target's class labels 
- Standardise the data to mean zero and variance one using an approapriate `sklearn` library (2.5 marks)   

(Total: 5 marks) 

In [14]:
#importing required libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#using an appropriate sklearn library we used in class to create y_train, y_test, X_train and X_test by splitting the data into 75% train and 25% test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3, stratify=y)

# Standardising the data to mean zero and variance one using an approapriate sklearn library
sc= StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

---
---

**Problem 4**. Support Vector Classifier and Accuracies (Total Marks: 30)


**Q1**. 

- Train a Support Vector Classifier on the standardised data (5 marks)
    - Use `rbf` kernel and set `random_state` to 24 (don't change any other parameters)
- Compute and print training and test dataset accuracies (5 marks)

(Total: 10 marks)   

In [15]:
# importing required libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# training the Support Vector Classifier using rbf kernel and set random_state to 24 (don't change any other parameters)
svc = SVC(kernel='rbf', random_state=24)
svc.fit(X_train_scaled, y_train)

# predictions on the training and test data
y_train_pred = svc.predict(X_train_scaled)
y_test_pred = svc.predict(X_test_scaled)

# computing training and test dataset accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# printing here 
print("Training accuracy of classifier without PCA:", train_accuracy)
print("Test accuracy of classifier without PCA:", test_accuracy)


Training accuracy of classifier without PCA: 0.8238222222222222
Test accuracy of classifier without PCA: 0.8133333333333334


<hr style="width:25%;margin-left:0;"> 

**Q2.**

- Extract 2 linear principal components from the standardised features using an appropriate `sklearn` library (5 marks)
- Train a Support Vector Classifier on the 2 principal components computed above (5 marks)   
    - Use `rbf` kernel and set `random_state` to 24 (don't change any other parameters)
- Compute and print training and test dataset accuracies (5 marks)


(Total: 15 marks)  

In [16]:
#importing required libraries
from sklearn.decomposition import PCA

# extracting PCA with 2 principal components from the standardised features using an appropriate sklearn library
pca = PCA(n_components=2)

# fitting PCA on the standardized training data and transform both training and test data
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# training the Support Vector Classifier with RBF kernel on the 2 principal components
svc_pca = SVC(kernel='rbf', random_state=24)
svc_pca.fit(X_train_pca, y_train)

# predictions on the training and test data
y_train_pred_pca = svc_pca.predict(X_train_pca)
y_test_pred_pca = svc_pca.predict(X_test_pca)

# computing training and test accuracies
train_accuracy_pca = accuracy_score(y_train, y_train_pred_pca)
test_accuracy_pca = accuracy_score(y_test, y_test_pred_pca)

# printing here
print("Training accuracy with PCA:", train_accuracy_pca)
print("Test accuracy with PCA:", test_accuracy_pca)


Training accuracy with PCA: 0.8014222222222223
Test accuracy with PCA: 0.8037333333333333


<hr style="width:25%;margin-left:0;"> 

**Q3**. 

- Comment on the suitability of the two classifiers to predict credit card defaults by commenting on (and comparing) the computed accuracies from the last two questions.


(Total: 5 marks)     


 ### Comments on the suitability of the two classifiers to predict credit card defaults by commenting on (and comparing) the computed accuracies from the last two questions:
 
 In evaluating the suitability of the two classifiers for predicting credit card defaults, one would consider their respective accuracies. The first classifier, trained on the full set of standardized features, might offer a comprehensive view, incorporating the full spectrum of available information. Its accuracy would reflect how well it can use a variety of data points to make a prediction.

On the other hand, the second classifier, trained on just two principal components derived from the same features, represents a simplified model. The principal components capture the directions of maximum variance in the data, but this reduction comes at the cost of losing some information. The accuracy of this classifier indicates how well it can predict defaults based on this condensed information.

A comparison of the accuracies of both classifiers can reveal whether the dimensionality reduction to two principal components retains enough information for accurate predictions. If the accuracies are close, it suggests that the two principal components capture most of the predictive variance in the data, making the second classifier (trained on PCA-transformed data) quite suitable despite its simplicity. However, if there's a significant drop in accuracy, this would imply that the dimensionality reduction oversimplified the model, leading to the loss of important predictive information.

Ultimately, the choice between the two models might also consider the trade-off between complexity (and the computational cost associated with it) and accuracy. If the simpler model's accuracy is adequate for the application's requirements, it might be preferred due to its efficiency. If accuracy is paramount, the more complex model might be the better choice, assuming the computational resources are available.


---
---

## Marking Criteria

To achieve a perfect score, your solutions must adhere to the criteria outlined below:

- Ensure that all numerical answers are accurate.
- Use the specific Python functions and libraries specified within the assignment instructions.
- For any written responses, provide accurate information, articulated in clear and complete sentences.
- Do not add extra cells beyond what is provided in the notebook.
- Do not print output with your code unless explicitly instructed to do so.
- Maintain a clean and organised notebook layout that is easy to follow.
    
---
---